In [16]:
%run 1_import_analysis.ipynb
%run 2_preprocessing.ipynb

'Cargando el dataset...'

'El dataset tiene 1599999 filas y 6 columnas'

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew



--------------------------------------------------

Corroborando valores nulos y duplicados
Nulls: 
target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

--------------------------------------------------

Duplicates:  0

--------------------------------------------------

Eliminando columnas innecesarias


'Distribución de la variable target:'

positive    800000
negative    799999
Name: target, dtype: int64


--------------------------------------------------

Train-test split:
X_train:  (1119999,)
X_test:  (480000,)
y_train:  (1119999,)
y_test:  (480000,)

--------------------------------------------------



## Modelo elegido: Logistic Regression

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from scipy.sparse import csr_matrix
from sklearn.preprocessing import FunctionTransformer


In [18]:


# Define the pipeline
pipeline = Pipeline([
      ('text_processing', TextProcessing()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sparse', FunctionTransformer(csr_matrix)),
    ('feature_selection', SelectKBest(score_func=chi2)),
    ('lr', LogisticRegression(max_iter=500))
])

# Define the grid search parameters
parameters = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__max_df': [0.75, 1.0],
    'tfidf__use_idf': [True, False],
    'feature_selection__k': [5000, 10000, 'all'],
    'lr__C': [0.1, 1, 10],
    'lr__penalty': ['none', 'l2'],
    'lr__solver': ['newton-cg',  'sag']
}

# Perform the grid search with cross-validation
lr_model = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=10, n_jobs=-1)
lr_model.fit(X_train, y_train)

# Print the best parameters
print("Best parameters: ")
print(lr_model.best_params_)


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


In [ ]:
## Models evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


print(lr_model.best_estimator_)
y_pred = lr_model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Classification report: ', classification_report(y_test, y_pred))
print('Confusion matrix: ', confusion_matrix(y_test, y_pred))


In [ ]:
import pickle

In [ ]:
chosen_model = lr_model.best_estimator_

In [ ]:
print('Saving model...')
separator()

filename = 'sentiment_analysis_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(chosen_model, file)